<a href="https://colab.research.google.com/github/mr-cri-spy/Movie_seat_enq_bot/blob/main/Crisbee_MovieBot_into_a_FastAPI_backend_04_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn nest-asyncio spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 126.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
# movie_bot_engine.py
import spacy
from bs4 import BeautifulSoup
import random

nlp = spacy.load("en_core_web_sm")

def extract_movie_city(text):
    doc = nlp(text)
    movie = city = None
    for ent in doc.ents:
        if ent.label_ == "GPE":
            city = city or ent.text
        elif ent.label_ in ["ORG", "WORK_OF_ART"]:
            movie = movie or ent.text
    return movie, city

def simulate_theatre_html(movie, city):
    theatres = ["Kingdome", "Inox Forum", "PVR Mall", "DRC Cinemas"]
    blocks = []
    for t in theatres:
        total = random.randint(80, 200)
        booked = random.randint(10, total - 20)
        available = total - booked
        html = f"""
        <div class="theatre">
            <h3>{movie} - {t}, {city}</h3>
            <div class="seats" data-total="{total}" data-booked="{booked}" data-available="{available}"></div>
        </div>
        """
        blocks.append(html)
    return "\n".join(blocks)

def parse_seats(html):
    soup = BeautifulSoup(html, "html.parser")
    theatres = soup.find_all("div", class_="theatre")
    parsed = []
    for t in theatres:
        name = t.find("h3").text
        seats = t.find("div", class_="seats")
        parsed.append({
            "theatre": name,
            "total": int(seats["data-total"]),
            "booked": int(seats["data-booked"]),
            "available": int(seats["data-available"])
        })
    return parsed



In [20]:
# app.py
from fastapi import FastAPI
from pydantic import BaseModel
from movie_bot_engine import extract_movie_city, simulate_theatre_html, parse_seats

app = FastAPI()

class Query(BaseModel):
    message: str

@app.post("/ask")
def ask_movie_bot(query: Query):
    movie, city = extract_movie_city(query.message)
    if not movie or not city:
        return {"error": "Movie or City not found in query."}

    html = simulate_theatre_html(movie, city)
    results = parse_seats(html)

    return {
        "movie": movie,
        "city": city,
        "results": results
    }


In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run("app:app", host="0.0.0.0", port=8000)


INFO:     Started server process [198]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print(" Public API:", public_url)


In [ ]:
from openai import OpenAI
from datetime import date

user_input = "Are seats available for Kingdome movie in Mysore today?"

# Prompt Template for LLM to extract info
prompt = f"""
Extract the following details:
- Movie Name
- City
- Date (Default to today if not specified)

User Input: {user_input}
"""

# (Mock response for now)
movie = "Kingdome"
city = "Mysore"
search_date = str(date.today())


In [ ]:
import requests
from bs4 import BeautifulSoup

city = "mysore"
url = f"https://in.bookmyshow.com/explore/movies-{city.lower()}/su-from-so"

try:
    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Try to find the movie in listings
    found_movies = soup.find_all("div", {"class": "card-title"})
    movie_found = False

    for movie_tag in found_movies:
        if movie.lower() in movie_tag.text.lower():
            movie_found = True
            break

    if movie_found:
        print(f" '{movie}' is currently showing in {city.title()}. Checking seats next...")
    else:
        print(f" '{movie}' not found in {city.title()}'s listing.")
except:
    print("  Failed to load the page. Maybe wrong city/movie name or HTML structure changed.")


In [9]:
# Simulating seat data (replace with real scraping later)
showtimes = ["10:00 AM", "1:30 PM", "7:30 PM"]
available_seats = [22, 15, 8]

for show, seats in zip(showtimes, available_seats):
    print(f" Show at {show} —  {seats} seats available")


 Show at 10:00 AM —  22 seats available
 Show at 1:30 PM —  15 seats available
 Show at 7:30 PM —  8 seats available


In [ ]:
# Write the movie_bot_engine code to a file
with open("movie_bot_engine.py", "w") as f:
    f.write("""
import spacy
from bs4 import BeautifulSoup
import random

nlp = spacy.load("en_core_web_sm")

def extract_movie_city(text):
    doc = nlp(text)
    movie = city = None
    for ent in doc.ents:
        if ent.label_ == "GPE":
            city = city or ent.text
        elif ent.label_ in ["ORG", "WORK_OF_ART"]:
            movie = movie or ent.text
    return movie, city

def simulate_theatre_html(movie, city):
    theatres = ["Kingdome", "Inox Forum", "PVR Mall", "DRC Cinemas"]
    blocks = []
    for t in theatres:
        total = random.randint(80, 200)
        booked = random.randint(10, total - 20)
        available = total - booked
        html = f'''
        <div class="theatre">
            <h3>{movie} - {t}, {city}</h3>
            <div class="seats" data-total="{total}" data-booked="{booked}" data-available="{available}"></div>
        </div>
        '''
        blocks.append(html)
    return "\\n".join(blocks)

def parse_seats(html):
    soup = BeautifulSoup(html, "html.parser")
    theatres = soup.find_all("div", class_="theatre")
    parsed = []
    for t in theatres:
        name = t.find("h3").text
        seats = t.find("div", class_="seats")
        parsed.append({
            "theatre": name,
            "total": int(seats["data-total"]),
            "booked": int(seats["data-booked"]),
            "available": int(seats["data-available"])
        })
    return parsed
""")